In [2]:
import json
import subprocess
import shutil
import os, glob
import time
from tqdm import tqdm
from shapely.geometry import Polygon, box

In [3]:
state = 'arkansas'

In [21]:
# We are keeping this files in drive
# name of the state we are working with
working_directory_path = "G:\\My Drive\\states\\{}".format(state)
processing             = "{}\\images\\processing".format(working_directory_path)
completed              = "{}\\images\\completed".format(working_directory_path)
filtered_files_path    = "{}\\counties_filtered".format(working_directory_path)

# We are keeping the images in box
img_path               = "C:\\Users\pjrio\\Box\\Images\\states\\{}".format(state)

# Path to some place with full acces to be able to delete files
temp                   = "C:\\Users\\pjrio\\OneDrive - University of Arkansas\\Documents"+"\\temp"

# Note: This paths might need to be change depending on where you installed the software
# paths for allmapsoft program
downloader_path        = "C:\\allmapsoft\\gsmd\\downloader.exe "
combiner_path          = "C:\\allmapsoft\\gsmd\\combiner.exe "

# Create images directory if it doesn't exist
os.makedirs(img_path, exist_ok=True)
os.makedirs(temp_path, exist_ok=True)
os.makedirs(processing, exist_ok=True)
os.makedirs(completed, exist_ok=True)

print("Paths:")
print("--Working directory   : " + working_directory_path)
print("--Filtered files      : " + filtered_files_path)
print("--Processing          : " + processing)
print("--Completed           : " + completed+"\n")

print("--Output directory    : " + img_path+"")
print("--Temporary files     : " + temp+"\n")

print("--Downloader          : " + downloader_path)
print("--Combiner            : " + combiner_path)

Paths:
--Working directory   : G:\My Drive\states\arkansas
--Filtered files      : G:\My Drive\states\arkansas\counties_filtered
--Processing          : G:\My Drive\states\arkansas\images\processing
--Completed           : G:\My Drive\states\arkansas\images\completed

--Output directory    : C:\Users\pjrio\Box\Images\states\arkansas
--Temporary files     : C:\Users\pjrio\OneDrive - University of Arkansas\Documents\temp

--Downloader          : C:\allmapsoft\gsmd\downloader.exe 
--Combiner            : C:\allmapsoft\gsmd\combiner.exe 


In [12]:
def get_square_bounds(polygon):
    # Convert the input polygon to a shapely Polygon
    shapely_polygon = Polygon(polygon[0])
    
    # Get the bounding box of the polygon
    bbox = shapely_polygon.bounds
    
    # Extract the left, right, top and bottom coordinates
    left, bottom, right, top = bbox
    return f"{left} {right} {top} {bottom}"

In [13]:
user = "Porfirio-desktop"

In [16]:
# Get a list of all the GeoJSON files in the directory
geojson_files = sorted(glob.glob(os.path.join(filtered_files_path, "*.geojson")), key=lambda x: os.path.getsize(x))
#print(len(geojson_files))
# Iterate through each file
for geojson_file in geojson_files:
    
    #print(geojson_file)
    county_name = os.path.basename(geojson_file).split('_')[2].split('.')[0] + "_County"
    print(f"Processing: {county_name}")

    processing_filepath = processing + "\\{}".format(county_name)+".txt"
    completed_filepath = completed + "\\{}".format(county_name)+".txt"
    
    # check if the file is being processed  
    if os.path.exists(completed_filepath):
        # check if you created the processing file
        # we do this in case there was an error 
        # and you did not finihsed processing the file
        # skip the file if you did not make it
        with open(completed_filepath, 'r') as p:
            p_user = p.read()
        p.close()
        print("**"+county_name +" was completed by: " + p_user)
        continue
    elif os.path.exists(processing_filepath):
        # check if you created the processing file
        # we do this in case there was an error 
        # and you did not finihsed processing the file
        # skip the file if you did not make it
        with open(processing_filepath, 'r') as p:
            p_user = p.read()
        p.close()
        if p_user != user:
            print("--"+county_name +" is being processed by: " + p_user)
            continue
    else:
        #create a temp file 
        with open(processing_filepath, 'w') as t:
            t.write(user)

    
    # Open the GeoJSON file
    with open(geojson_file, 'r') as f:
        data = json.load(f)
    
    dst  = os.path.join(img_path, county_name)
    # Create images directory if it doesn't exist
    os.makedirs(dst, exist_ok=True)
    
    # Get the checkpoint file for this GeoJSON file
    checkpoint_file = working_directory_path +"\\images\\{}\\checkpoint.txt".format(county_name)
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            start_index = int(f.readline().strip())
            print("Processed images: " + str(start_index))
    else:
        start_index = 0

    # Iterate through the features in the GeoJSON file
    for num, feature in enumerate(tqdm(data['features'][start_index:], desc='Processing data')):
        # Get the coordinates of the current feature
        coordinates = feature['geometry']['coordinates']
        i = num + start_index

        # Format = [left longitude, right longitude, top latitude, bottom latitude]
        square = get_square_bounds(coordinates)

        filename = f"{county_name}_task{i}.jpg"

        # Generate command
        gmid_name = county_name + "_" + f"task{i}.gmid"
        para2 = "1"
        para3 = "21"
        command = downloader_path + '"'+gmid_name +'\" ' + para2 +" "+ para3 + " "+ square +' "'+ temp +'"'
        # Call cmd and run downloader program
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        combine = combiner_path +'"' + temp +"\\"+ gmid_name +"\" jpg meters"

        # Call cmd and run combine program
        subprocess.run(combine, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Set the source and destination directories
        name = county_name + f"_task{i}"
        img  = name + ".jpg"
        src  = os.path.join(temp, name+"_combined")

        # Set the source and destination file paths
        src_file = os.path.join(src, img)
        dst_file = os.path.join(dst, filename)

        # Use the shutil module to move the file
        shutil.move(src_file, dst_file)

        # Clean
        # -----Generate command to delete path
        folder_path = os.path.join(temp, name)
        combined_folder_path = os.path.join(temp, name+"_combined")

        # -----Use the remove function to delete the path
        shutil.rmtree(folder_path)
        shutil.rmtree(combined_folder_path)

        # Use glob to get a list of all the files in the directory
        files   = glob.glob(os.path.join(temp, '*'))

        # Iterate over the list of files and delete them one by one
        for file in files:
            if os.path.isfile(file):
                # If it's a regular file, delete it
                os.remove(file)
            # Save the checkpoint
        with open(checkpoint_file, 'w') as f:
            f.write(str(num + start_index))
    os.remove(processing_filepath)
    with open(completed_filepath, 'w') as t:
            t.write(user)

Processing: Calhoun_County
**Calhoun_County was completed by: Porfirio-desktop
Processing: Dallas_County
**Dallas_County was completed by: Porfirio-desktop
Processing: Lee_County
**Lee_County was completed by: Porfirio-desktop
Processing: Monroe_County
**Monroe_County was completed by: TAs_office-Porfirios_desktop
Processing: Woodruff_County
**Woodruff_County was completed by: Porfirio-laptop2
Processing: Cleveland_County
**Cleveland_County was completed by: TAs_office-Porfirios_desktop
Processing: Lafayette_County
**Lafayette_County was completed by: Porfirio-laptop2
Processing: Prairie_County
**Prairie_County was completed by: TAs_office-Porfirios_desktop
Processing: Nevada_County
**Nevada_County was completed by: CSCE-laptop
Processing: Lincoln_County
**Lincoln_County was completed by: TAs_office-Porfirios_desktop
Processing: Desha_County
--Desha_County is being processed by: Porfirio-laptop2
Processing: Perry_County
Processed images: 899


Processing data:   0%|          | 9/6188 [01:22<15:44:03,  9.17s/it]


KeyboardInterrupt: 